# Search Algorithms

In [ ]:
import os
from overrides import overrides
from typing import List
import torch
from torch import nn
import numpy as np
import json
from random import Random

from archai.discrete_search.api import ArchaiModel, EvolutionarySearchSpace, BayesOptSearchSpace

We will re-use the CNN search space created in the [search space example](./search_space.ipynb).

In [ ]:
import nbimporter
from search_space import MyModel, CNNSearchSpaceExt as CNNSearchSpace

In [ ]:
ss = CNNSearchSpace(max_layers=10, kernel_list=[3, 5, 7], hidden_list=[32, 64, 128])

In [ ]:
m = ss.random_sample()
m

## Dataset Provider

Datasets are represented in Archai throught the [`DatasetProvider`](../../reference/api/archai.discrete_search.api.rst) class. For this example, we will use the built-in dataset provider of the MNIST dataset.

In [ ]:
from archai.datasets.cv.mnist_dataset_provider import MnistDatasetProvider

In [ ]:
dataset_provider = MnistDatasetProvider()

We can get train/test PyTorch datasets from a DatasetProvider by calling `dataset_provider.get_datasets(load_train, load_test, transforms_train, transforms_test)`

In [ ]:
# Loads only the training set
tr_d = dataset_provider.get_train_dataset()

## Wrapping custom evaluation code

We will evaluate our models using partial trainig validation accuracy as a proxy for final task performance. 

In [ ]:
from archai.api.dataset_provider import DatasetProvider
from archai.discrete_search.api import ModelEvaluator
from archai.discrete_search.evaluators import RayParallelEvaluator

from tqdm import tqdm
import math


class PartialTrainingValAccuracy(ModelEvaluator):
    def __init__(self, dataset: DatasetProvider, training_epochs: float = 1.0, lr: float = 1e-4, device: str = 'cpu',
                 progress_bar: bool = False):
        self.training_epochs = training_epochs
        self.dataset_provider = dataset
        self.device = device
        self.lr = lr
        self.progress_bar = progress_bar
    
    @overrides
    def evaluate(self, model, budget = None) -> float:
        # Loads the dataset
        tr_data = self.dataset_provider.get_train_dataset()
        val_data = self.dataset_provider.get_val_dataset()
        
        tr_dl = torch.utils.data.DataLoader(tr_data, batch_size=16, shuffle=True, num_workers=4)
        val_dl = torch.utils.data.DataLoader(val_data, batch_size=16, shuffle=False, num_workers=4)
        
        # Training settings
        optimizer = torch.optim.Adam(model.arch.parameters(), lr=self.lr)
        criterion = nn.CrossEntropyLoss()
        
        model.arch.train()
        model.arch.to(self.device)
        
        # Partial training
        epoch_iter = range(math.ceil(self.training_epochs))
        if self.progress_bar:
            epoch_iter = tqdm(epoch_iter, desc=f'Training model {model.archid}')

        for epoch_nb in epoch_iter:
            # Early stops for fractional values of training epochs (e.g, 0.2)
            early_stop = len(tr_dl) + 1
            if 0 < (self.training_epochs - epoch_nb) < 1:
                early_stop = int((self.training_epochs - epoch_nb) * len(tr_dl))
            
            for i, (x, y) in enumerate(tr_dl):
                if i >= early_stop:
                    break
                
                optimizer.zero_grad()
                
                pred = model.arch(x.to(self.device))
                loss = criterion(pred, y.to(self.device))

                loss.backward()
                optimizer.step()
        
        # Evaluates final model
        model.arch.eval()
        
        with torch.no_grad():
            val_pred, val_target = [], []

            for x, y in val_dl:
                val_pred.append(model.arch(x.to(self.device)).argmax(axis=1).to('cpu'))
                val_target.append(y.to('cpu'))

            val_pred, val_target = torch.cat(val_pred, axis=0), torch.cat(val_target, axis=0)
            val_acc = (val_pred.squeeze() == val_target.squeeze()).numpy().mean()

        # Returns model to cpu
        model.arch.cpu()
        
        return val_acc

Let's test our evaluator:

In [ ]:
partial_tr = PartialTrainingValAccuracy(
    dataset_provider,
    training_epochs=0.001, # Trains for 1/1000 of an epoch
    progress_bar=True
)

In [ ]:
partial_tr.evaluate(ss.random_sample())

We can make this objective more efficient evaluating multiple architectures in parallel. To do that, we can use the `RayParallelObjective` wrapper mentioned in the [previous example](./evaluators.ipynb):

In [ ]:
parallel_partial_tr = RayParallelEvaluator(partial_tr)

Let's test our partial training objective sending two random architectures

In [ ]:
# NBVAL_SKIP
parallel_partial_tr.send(ss.random_sample())
parallel_partial_tr.send(ss.random_sample())

In [ ]:
# NBVAL_SKIP
parallel_partial_tr.fetch_all()

To run the same objective distributing jobs across multiple GPUs, just set the `num_gpus` parameter from [ray.init](https://docs.ray.io/en/latest/ray-core/package-ref.html#ray-init) and set `device='cuda'` (This assumes you have installed the NVidia CUDA SDK and PyTorch for CUDA as per the setup instructions at https://pytorch.org/get-started/locally/)

```python
RayParallelObjective(
    PartialTrainingValAccuracy(training_epochs=1, device='cuda'),
    num_gpus=0.5, # 2 jobs per gpu available
    max_calls=1
)
```

## Defining Search Objectives

Search optimization objectives are specified using the `archai.discrete_search.SearchObjectives` class

In [ ]:
from archai.discrete_search.api import SearchObjectives

objectives = SearchObjectives()

### Adding objectives

To add search objectives, we can use the `SearchObjectives.add_objective` method


In [ ]:
from archai.discrete_search.evaluators import AvgOnnxLatency, TorchFlops

objectives.add_objective(
    # Objective function name (will be used in plots and reports)
    name='ONNX Latency (ms)',  
    
    # ModelEvaluator object that will be used to evaluate the model
    model_evaluator=AvgOnnxLatency(input_shape=(1, 1, 28, 28), num_trials=3),  
    
    # Optimization direction, `True` for maximization or `False` for minimization
    higher_is_better=False,

    # Whether this objective should be considered 'compute intensive' or not.
    compute_intensive=False 
)


The `compute_intensive` flag is used in some search algorithms to help increase search efficiency. For instance, search algorithms that use surrogate models may try to estimate the value of expensive objective functions of unseen architectures in certain situations, while cheap objectives (`compute_intensive=False`) will just be computed directly.

In [ ]:
objectives.add_objective(
    'FLOPs', TorchFlops(torch.randn(1, 1, 28, 28)),
    higher_is_better=False,
    compute_intensive=False,
    # We may optionally add a constraint. 
    # Architectures outside this range will be ignored by the search algorithm
    constraint=(0.0, 1e9)
)

Additionally, objectives that are cheap to evaluate (`compute_intensive=False`) may receive an optional `constraint` argument. Model candidates outside this range will
be ignored by the search algorithm.

We can evaluate cheap objectives calling `SearchObjectives.eval_cheap_objs(model_list)`

In [ ]:
samples = [ss.random_sample() for _ in range(2)]
objectives.eval_cheap_objs(samples,
                           progress_bar=True)

We can check if a model satisfies the constraints we added for the FLOPs objective by calling `SearchObjectives.validate_constraints(model_list)` or `SearchObjectives.is_model_valid(ss.random_sample())`

In [ ]:
m = ss.random_sample()

objectives.validate_constraints([m])

In [ ]:
objectives.is_model_valid(m)

By default, all objective and constraints evaluations are cached to prevent spending resources in the same architecture twice.

In [ ]:
# The evaluation cache is built using the
# tuple (obj_name, archid, budget)
objectives.lookup_cache('FLOPs', samples[0].archid, None)

Caching can be disabled setting `SearchObjectives(cache_objective_evaluation=False)`.

Now, let's try adding the partial training objective we created before. 

<div class="alert alert-block alert-warning">
The code below requires a GPU compatible with CUDA. If running on CPU, make sure to change `device='cuda'` to `device='cpu'` and set the `num_cpus` parameter for `RayParallelEvaluator` accordingly.
</div>

In [ ]:
objectives.add_objective(
    'Partial training Validation Accuracy (1 epoch)',
    RayParallelEvaluator(
        PartialTrainingValAccuracy(dataset_provider, training_epochs=1, device='cuda'),
        num_gpus=0.5, # 2 jobs per gpu available
        max_calls=1
    ),
    higher_is_better=True,
    compute_intensive=True # This is a compute intensive evaluator
)

Expensive objectives can be evaluated using `SearchObjectives.eval_expensive_objs(model_list)`

Alternatively, all objectives (expensive and cheap) can also be evaluated using `SearchObjectives.eval_all_objs`.

### Adding extra constraints

Besides the constraint parameter from cheap objectives, it is also possible to add extra constraints that are not search objectives (and thus should not be optimized by NAS algorithms).

In [ ]:
from archai.discrete_search.evaluators import TorchNumParameters

objectives.add_constraint(
    'Number of parameters',
    TorchNumParameters(),
    constraint=(0.0, 1e6)
)

In [ ]:
objectives.validate_constraints([m])

In [ ]:
objectives.is_model_valid(m)

## Using a search algorithm

Now that we know how to create and use search objectives, we can finally use a search algorithm do to Neural Architecture Search!

### Example: `EvolutionParetoSearch`

Let's start with an evolutionary-based search algorithm

In [ ]:
from archai.discrete_search.algos import EvolutionParetoSearch

In [ ]:
algo = EvolutionParetoSearch(
    ss, objectives, 
    output_dir='./out_evo',
    num_iters=5, num_crossovers=5,
    mutations_per_parent=2,
    max_unseen_population=10,
    seed=42
)

In [ ]:
# NBVAL_SKIP
search_results = algo.search()

By default all algorithms will save the final pareto architectures `{output_dir}/pareto_models_iter_*/`, pareto evolution plots `pareto_*.png` and search state tables with all the results `{output_dir}/search_state_*.csv`

In [ ]:
# NBVAL_SKIP
os.listdir('./out_evo')

It is also possible to get information from the `search_results` object directly:

In [ ]:
# NBVAL_SKIP
search_results.plot_2d_pareto_evolution(('ONNX Latency (ms)', 'Partial training Validation Accuracy (1 epoch)'))

We can get `pandas.DataFrame` object with the search results calling

In [ ]:
# NBVAL_SKIP
results_df = search_results.get_search_state_df()

In [ ]:
# NBVAL_SKIP
results_df.query('is_pareto').sort_values('Partial training Validation Accuracy (1 epoch)')

Since our search space is also compatible with Bayesian Optimization algorithms, let's try more sophisticated algorithm like MO-BANANAS. 

MO-BANANAS will progressively train a surrogate model based on the data gathered during search. This surrogate model will be used to predict the result of expensive objective function evaluations and will try to determine what are the best possible architectures according to the surrogate model.

In [ ]:
from archai.discrete_search.algos import MoBananasSearch

In [ ]:
algo2 = MoBananasSearch(
    ss, objectives, 
    output_dir='./out_bananas', 
    num_iters=5, mutations_per_parent=5,
    num_candidates=20,
    seed=43
)

In [ ]:
# NBVAL_SKIP
search_results2 = algo2.search()

In [ ]:
# NBVAL_SKIP
os.listdir('./out_bananas')

In [ ]:
# NBVAL_SKIP
search_results2.plot_2d_pareto_evolution(('ONNX Latency (ms)', 'Partial training Validation Accuracy (1 epoch)'))

MO-BANANAS will also save the predictive mean and variance of the expensive objectives during that iteration .

In [ ]:
# NBVAL_SKIP
results_df2 = search_results2.get_search_state_df()
results_df2.query('is_pareto').sort_values('Partial training Validation Accuracy (1 epoch)')

Let's use [plotly](https://plotly.com/) to compare the final pareto frontiers of both algorithms:

In [ ]:
# NBVAL_SKIP
import pandas as pd
import plotly.express as px

merged_results_df = pd.concat([
    results_df.assign(algo='Evolution Pareto'),
    results_df2.assign(algo='Mo-BANANAS')
], axis=0)

fig = px.scatter(
    merged_results_df.query('is_pareto'), 
    'ONNX Latency (ms)', 
    'Partial training Validation Accuracy (1 epoch)',
    hover_name='archid',
    color='algo',
    facet_col='algo'
)

fig.layout = fig.layout.update(showlegend=False)
fig

In this particular example both algorithms found similar pareto frontiers.